# Experiment 3 — Metadata Combinations

This notebook aggregates **Experiment 3** results into tables (no figures).  

1. Gender
2. Duration (log)
3. Item
4. Gender + Duration (log)
5. Gender + Item + Duration (log)


In [ ]:
import os, pandas as pd, numpy as np
from pathlib import Path

BASE = '/Users/susie/Desktop/Multimodal analysis of disagreement/disagreement-dataset'
EXP3_ROOT = os.path.join(os.path.dirname(BASE), 'Results', 'exp3')
FIG_DIR = os.path.join(os.path.dirname(BASE), 'Figures&Graphs')
Path(FIG_DIR).mkdir(parents=True, exist_ok=True)

display_map = {
    'gender': 'Gender',
    'duration': 'Duration (log)',
    'item': 'Item',
    'gender_duration': 'Gender + Duration (log)',
    'gender_item_duration': 'Gender + Item + Duration (log)',
}
order = ['Gender','Duration (log)','Item','Gender + Duration (log)','Gender + Item + Duration (log)']

rows = []
if not os.path.isdir(EXP3_ROOT):
    raise SystemExit(f'Not found: {EXP3_ROOT}. 请先运行 exp3_meta_combo.py')

for sub in sorted(os.listdir(EXP3_ROOT)):
    p = os.path.join(EXP3_ROOT, sub, 'summary.csv')
    if not os.path.exists(p):
        continue
    df = pd.read_csv(p)

    inject = 'unknown'
    if 'Mode' in df.columns and isinstance(df.at[0, 'Mode'], str):
        part = df.at[0, 'Mode']
        if '_meta_clip' in part:
            inject = 'clip'
        elif '_meta_turn' in part:
            inject = 'turn'

    label = display_map.get(sub, sub.replace('_', ' ').title())
    df['Combo'] = label
    df['Inject'] = inject
    rows.append(df)

if not rows:
    raise SystemExit('No summaries found under Results/exp3/*/summary.csv')

agg = pd.concat(rows, ignore_index=True)

keep = ['Combo','Inject','Mode','Meta','MSE_mean','RMSE_mean','MAE_mean','R2_mean','r_mean','p_mean','r_overall','p_overall']
for c in keep:
    if c not in agg.columns:
        agg[c] = np.nan

agg['Combo'] = pd.Categorical(agg['Combo'], categories=order, ordered=True)
agg = agg.sort_values(['Inject','Combo']).reset_index(drop=True)

for inj in ['clip','turn']:
    sub_df = agg[agg['Inject']==inj].copy()
    if not sub_df.empty:
        out_csv = os.path.join(FIG_DIR, f'ablation_metrics_exp3_{inj}.csv')
        sub_df[keep].to_csv(out_csv, index=False)
        print('Saved table to:', out_csv)
        display(sub_df[keep])
    else:
        print(f'[INFO] no rows for Inject={inj}, skipped.')

Saved table to: /Users/susie/Desktop/Multimodal analysis of disagreement/Figures&Graphs/ablation_metrics_exp3_clip.csv


,Combo,Inject,Mode,Meta,MSE_mean,RMSE_mean,MAE_mean,R2_mean,r_mean,p_mean,r_overall,p_overall
0,Gender,clip,hybrid_meta_clip,Gender,108.504234,10.362886,7.979146,0.464926,0.713476,3.351767e-18,0.695858,3.913935e-100
1,Duration (log),clip,hybrid_meta_clip,Duration,110.869179,10.446119,7.930525,0.458730,0.722103,1.757787e-18,0.691414,2.264929e-98
2,Item,clip,hybrid_meta_clip,Item,118.941696,10.729655,8.162481,0.429066,0.716514,6.291618e-19,0.667237,2.564153e-89
3,Gender + Duration (log),clip,hybrid_meta_clip,"Gender,Duration",101.687790,10.033015,7.767130,0.496364,0.724933,2.969811e-20,0.722170,2.859989e-111
4,Gender + Item + Duration (log),clip,hybrid_meta_clip,"Gender,Item,Duration",102.167137,10.052835,7.714896,0.493385,0.720917,1.133381e-16,0.714706,5.543326e-108


[INFO] no rows for Inject=turn, skipped.
